## Extracting data from logfiles

In [52]:
from log_parser import parse_keyboard_log
from log_parser import parse_mouse_log
from keyboard_heatmap import KeyboardHeatmap

# Segment length of each data sample
seg_length = 100

routing_file = open('.routing', 'r')
Lines = routing_file.readlines()

# List of parsed logfiles
keyboard = []
mouse = []

for line in Lines:
    line = line.strip()
    if 'keyboard_actions.log' in line:
        keyboard.append(parse_keyboard_log(line))
    '''elif 'mouse_actions.log' in line:
        mouse.append(parse_mouse_log(line))'''

# Split into train and test sets
X_train = []
X_test = []
Y_train = []
Y_test = []

for k in range(len(keyboard) - 1):
    for i in range(int(keyboard[k].time.iloc[-1] / seg_length)):
        # For each segment in each logfile
        # Create a heatmap for that segment
        heatmap = KeyboardHeatmap(keyboard[k], i, seg_length)
        bheatmap = heatmap.to_binary_class_label("MITSOL")
        # If the heatmap isn't blank
        if bheatmap.class_label() != 'Null':
            X_train.append(bheatmap.heatmap_data().ravel().tolist())
            Y_train.append(bheatmap.class_label())

for k in range(len(keyboard) - 1,len(keyboard)):
    for i in range(int(keyboard[k].time.iloc[-1] / seg_length)):
        # For each segment in each logfile
        # Create a heatmap for that segment
        heatmap = KeyboardHeatmap(keyboard[k], i, seg_length)
        bheatmap = heatmap.to_binary_class_label("MITSOL")
        # If the heatmap isn't blank
        if bheatmap.class_label() != 'Null':
            X_test.append(bheatmap.heatmap_data().ravel().tolist())
            Y_test.append(bheatmap.class_label())

print(f"Train samples: {len(Y_train)}")
print(f"Test samples: {len(Y_test)}")

Train samples: 145
Test samples: 36


## Training a ANN model on keyboard heatmap data

In [66]:
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import random

# Make random_state random each run, it causes different data shuffel each run.
# Make it a constant if want to test activation funcyion and/or solver
randomState = random.randint(0,10000)

# Split into train and test sets. 70% for traning data, 30% for test data
#X_train, X_test, Y_train, Y_test = train_test_split(X_actual, Y_actual, random_state=randomState, test_size=0.3)

# Normalize the data to make it easier and faster for MLP to converge
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



# Activation: 'relu' and 'logistic' are considered useful. But their performances are very similar now.
# Solver: 'adam' is better for dataset with thousands of samples, 'lbfgs' is much faster for small dataset
#         But 'adam' performs better in our case.

mlp = MLPClassifier(hidden_layer_sizes=(100,100), activation='relu', solver='adam', max_iter=10000)
mlp.fit(X_train,Y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

#Show model prediction and evaluation
from sklearn.metrics import classification_report,confusion_matrix
print("Result on Training data")
print(confusion_matrix(Y_train,predict_train))
print(classification_report(Y_train,predict_train))
print("Result on Test data")
print(confusion_matrix(Y_test,predict_test))
print(classification_report(Y_test,predict_test))



Result on Training data
[[109   0]
 [  0  36]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       109
           1       1.00      1.00      1.00        36

    accuracy                           1.00       145
   macro avg       1.00      1.00      1.00       145
weighted avg       1.00      1.00      1.00       145

Result on Test data
[[11 25]
 [ 0  0]]
              precision    recall  f1-score   support

           0       1.00      0.31      0.47        36
           1       0.00      0.00      0.00         0

    accuracy                           0.31        36
   macro avg       0.50      0.15      0.23        36
weighted avg       1.00      0.31      0.47        36



C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
